In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)#pd.reset_option(“max_columns”)
import matplotlib.pyplot as plt

# ICU -- ICU patient measure data

In [2]:
print("ICU Patient Dataset:")
icu = pd.read_csv('bsi_old_deidentified_final.csv')
icu = icu.astype({"tsa": int})
icu = icu.astype({"id": int})
print("Rows and columns:", icu.shape)
print("Admissions: ", len(icu['id'].unique()))

ICU Patient Dataset:
Rows and columns: (5116484, 113)
Admissions:  9954


# BCO -- blood culture obtained patient data

In [3]:
print("Blood Culture Obtained Patient Dataset:")
bco = pd.read_csv('transplant_bsi_deidentified.csv')
bco = bco.astype({"tsa": int})
bco = bco.astype({"ID": int})
print("Rows and columns:", bco.shape)
print("Admissions: ", len(bco['ID'].unique()))

Blood Culture Obtained Patient Dataset:
Rows and columns: (15577, 25)
Admissions:  5671


# Round BCO time

In [4]:
# add a column 'tsa_round_15' to bco dataset for rounded time
bco['tsa_bc_15'] = np.nan

for N in list(bco['ID'].unique()):
    # get the first time point of the patient since addmission
    start = icu.loc[icu['id']==N,['tsa']].iloc[0]
    # get the old time STAMPS
    tsa = bco.loc[bco['ID']==N,['tsa']]
    # round it in every 15 minute
    tsa_15 = (tsa-start)//15*15+start
    # assign it back to old dataset
    bco.loc[bco['ID']==N,['tsa_bc_15']] = tsa_15['tsa']

bco = bco.astype({"tsa_bc_15": int})

In [5]:
print("Blood Culture Obtained Patient Dataset after rounding time:")
print("Rows and columns:", bco.shape)
print("Admissions: ", len(bco['ID'].unique()))

Blood Culture Obtained Patient Dataset after rounding time:
Rows and columns: (15577, 26)
Admissions:  5671


## create column for true positive labels

In [6]:
txp_id = list(bco.loc[bco['Confirmed Txp']=='yes','ID'].unique())
infection_list = ['GP','GN','fungus','mycobacteria']
# record the number of types of infection in 'Ture_positive_n'
bco['True_positive_n'] = bco[infection_list].sum(axis=1)
# create one hot label 'True_positive'
bco['True_positive'] = 0
bco.loc[bco['True_positive_n'] >0,['True_positive']] = 1
# save new dataset
bco.to_csv('bco.csv',index=False)

# Outer merge ICU & BCO dataset

In [7]:
bco = pd.read_csv('bco.csv')

In [8]:
icu_bco = pd.merge(left=icu,
                   right=bco,
                   left_on=['id','tsa'],
                   right_on=['ID','tsa_bc_15'],
                   how='outer',
                   suffixes=[None,'_bc'],
                   copy = True)
# fulfill empty keys caused by outer merging
icu_bco['tsa'] = icu_bco['tsa'].fillna(icu_bco['tsa_bc_15'])
icu_bco['id'] = icu_bco['id'].fillna(icu_bco['ID'])
icu_bco = icu_bco.groupby('id').apply(lambda x:x.sort_values('tsa')).reset_index(drop=True)

In [9]:
icu_bco = icu_bco.groupby('id').apply(lambda x:x.sort_values('tsa')).reset_index(drop=True)

In [10]:
print("whole ICU patient dataset with blood culture result if exist:")
print("Rows and columns:", icu_bco.shape)
print("Admissions: ", len(icu_bco['id'].unique()))
icu_bco.head(10)

whole ICU patient dataset with blood culture result if exist:
Rows and columns: (5124891, 141)
Admissions:  9954


,bid,hr,rr,so2,x.hr.rr,x.hr.so2,x.rr.so2,s.rr,s.so2,rsa,edrk,edr,x.hr.edr,mrr,srr,af,cosen,dfa,ds,lds,hem.s.1,hem.s,hem.m,int.s.amani,int.s,int.m,sep.s,sep.m,s.edrk,n.edrk,s.edr,pt.rr,s.pt.rr,s.hr,pdel,s.pdel,x.ptt.ptt,x.rri.ptt,te.rri.ptt,SODIUM,POTASSIUM,CHLORIDE,CO2,BLOOD.UREA.NITROGEN,CREATININE,GLUCOSE,CALCIUM,WHITE.BLOOD.CELL.COUNT,HEMOGLOBIN,HEMATOCRIT,PLATELET.COUNT,NEUTROPHILS.PERCENT,AST.GOT,ALT.GPT,TOTAL.PROTEIN,ALBUMIN,TOTAL.BILIRUBIN,ALKALINE.PHOSPHATASE,MAGNESIUM,PHOSPHORUS,OXYGEN.SATURATION,PARTIAL.THROMBOPLASTIN.TIME,PROTIME,PROTIME.INR,PH.ARTERIAL,PCO2,PO2,BICARBONATE,BASE.EXCESS,FIO2,TROPONIN.I,LACTIC.ACID,havelabs,haveabg,Resp,SpO2,Pulse,SBP,O2.Flow,Temp,GCS,PEEP,FiO2,Voided.Urine,DBP,age,unit,bps,bpd,pt.used,crm,SIRS,CCI,MI,CHF,PVD,CVD,DEM,CPD,RHM,MLD,DNC,DWC,REN,CAN,LIV,MET,HIV,APACHE,nline,tt.line,tsa,id,instance_id,sets_per_instance,growths_per_instance,pathogens_per_instance,possible_contaminants_per_instance,confirmed_contaminants_per_instance,set_results,Instance_Verdict,GP_isolate_n,GN_isolate_n,fungus_isolate_n,mycobacteria_isolate_n,all_isolate_n,GP,GN,fungus,mycobacteria,polymicrobial,Transplant Status,Confirmed Txp,SOT,HSCT,Transplant AdmissionDiff,tsa_bc,ID,tsa_bc_15,True_positive_n,True_positive
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,205.0,1.0,9974.0,2.0,0.0,0.0,0.0,0.0,"c(""NA"", ""NA"")",No Growths,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,215.0,1.0,205.0,0.0,0.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.0,100.0,95.0,122.0,NaN,NaN,NaN,NaN,NaN,NaN,57.0,NaN,ER,NaN,NaN,NaN,False,2.0,3.0,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,2.0,0.0,NaN,250.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,135.4,5.53,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.7,43.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.0,100.0,95.0,122.0,NaN,NaN,NaN,NaN,NaN,NaN,57.0,NaN,ER,NaN,NaN,NaN,False,2.0,3.0,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,3.0,0.0,NaN,295.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,135.4,5.53,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.7,43.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.0,100.0,102.0,124.0,NaN,NaN,NaN,NaN,NaN,NaN,63.0,NaN,ER,NaN,NaN,NaN,False,2.0,3.0,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,1.0,0.0,NaN,325.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,135.4,5.53,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.7,43.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.0,100.0,100.0,125.0

In [11]:
icu_bco.to_csv('uva_merged.csv', index = False)